In [1]:
# 1. Load environment variables and create the SQLAlchemy engine
import os
from dotenv import load_dotenv
from sqlalchemy import create_engine
import pandas as pd
from datetime import date, timedelta
import requests, json
from sqlalchemy import text
from pathlib import Path

In [2]:
# Load .env from working directory
load_dotenv('.env')

True

In [3]:
# PostGIS connection vars
POSTGIS_USER = os.getenv('POSTGRES_USER')
POSTGIS_PASS = os.getenv('POSTGRES_PASSWORD')
POSTGIS_HOST = os.getenv('POSTGRES_HOST')
POSTGIS_PORT = os.getenv('POSTGRES_PORT')
POSTGIS_DB   = os.getenv('POSTGRES_DB')

# Create the engine
engine = create_engine(
    f"postgresql+psycopg2://"
    f"{POSTGIS_USER}:{POSTGIS_PASS}"
    f"@{POSTGIS_HOST}:{POSTGIS_PORT}/{POSTGIS_DB}"
)

In [4]:
url = "http://localhost:11434/api/chat"

In [5]:
# 1) Define the last 3 full days (excluding today)
end_exclusive = date.today()
start_date   = end_exclusive - timedelta(days=3)   # 3 days before today
date_list    = [start_date + timedelta(days=i) for i in range(3)]  
today = date.today()

# check
print(date_list)

[datetime.date(2025, 7, 9), datetime.date(2025, 7, 10), datetime.date(2025, 7, 11)]


In [6]:
prompt_template = """
Here are the SF 311 metrics for {date}:

**Request Type Breakdown**  
{request_table}

**Average resolution time:** {avg_time}  
**Open-case aging:** {aging}

Please write a concise, dashboard-style report that follows this structure exactly:

1. **Summary:** “{open_count} open vs. {closed_count} closed cases.”  
2. **Top 3 Request Types:** A bulleted list of the three request types.
3. **Closing:** One sentence commenting on the resolution speed ({avg_time}) and the open-case aging distribution ({aging}).

Use only the numbers shown above; do not add any extra interpretation. Keep it to 5–7 lines of plain text.
""".strip()

In [7]:
# 1) Find the last non-empty date
last_dt = None
for dt in reversed(date_list):
    count = pd.read_sql(f"""
        SELECT COUNT(*) AS cnt
          FROM castro_311
         WHERE opened_ts::date = '{dt.isoformat()}';
    """, engine).iloc[0, 0]
    if count > 0:
        last_dt = dt
        break

# 2) If none found, just write the header and exit
if last_dt is None:
    md_lines = ["# SF Castro 311 Dashboard Summary (no data)", ""]
else:
    dt_str = last_dt.isoformat()
    md_lines = [f"# SF Castro 311 Dashboard Summary ({dt_str})", ""]

    # a) Pull status counts
    df_status = pd.read_sql(f"""
        SELECT status, COUNT(*) AS count
          FROM castro_311
         WHERE opened_ts::date = '{dt_str}'
         GROUP BY status;
    """, engine)
    open_count   = int(df_status.loc[df_status.status == 'Open',   'count'].sum() or 0)
    closed_count = int(df_status.loc[df_status.status == 'Closed', 'count'].sum() or 0)
    total = open_count + closed_count

    # b) Pull request_type counts
    df_req = pd.read_sql(f"""
        SELECT request_type, COUNT(*) AS count
          FROM castro_311
         WHERE opened_ts::date = '{dt_str}'
         GROUP BY request_type
         ORDER BY count DESC;
    """, engine)

    # c) Average resolution time
    avg_hours = pd.read_sql(f"""
        SELECT AVG(EXTRACT(EPOCH FROM (closed_ts - opened_ts)))/3600.0 AS avg_hours
          FROM castro_311
         WHERE opened_ts::date = '{dt_str}'
           AND closed_ts IS NOT NULL;
    """, engine).iloc[0,0] or 0
    avg_str = f"{avg_hours:.1f} hrs"

    # d) Aging buckets
    a0, a1, a2 = pd.read_sql(f"""
        SELECT
          COUNT(*) FILTER (WHERE now() - opened_ts <= INTERVAL '1 day')          AS bucket_0_1,
          COUNT(*) FILTER (WHERE now() - opened_ts >  INTERVAL '1 day'
                         AND now() - opened_ts <= INTERVAL '3 days')            AS bucket_1_3,
          COUNT(*) FILTER (WHERE now() - opened_ts >  INTERVAL '3 days')         AS bucket_3_plus
        FROM castro_311
       WHERE opened_ts::date = '{dt_str}'
         AND closed_ts IS NULL;
    """, engine).iloc[0]
    aging_str = f"0–1 d={a0}, 1–3 d={a1}, 3+ d={a2}"

    # e) Build raw-metrics block
    req_table = df_req.to_markdown(index=False) if not df_req.empty else "*no request types*"
    md_lines += [
        f"## {dt_str}",
        "",
        "### Raw Metrics",
        "",
        f"• Total cases: {total}  ",
        f"• Open: {open_count}  ",
        f"• Closed: {closed_count}  ",
        "",
        "**Request Type Breakdown:**",
        "",
        req_table,
        "",
        f"**Average resolution time:** {avg_str}  ",
        f"**Open-case aging:** {aging_str}",
        ""
    ]

    # f) AI Report
    prompt = prompt_template.format(
        date=dt_str,
        total=total,
        open_count=open_count,
        closed_count=closed_count,
        request_table=req_table,
        avg_time=avg_str,
        aging=aging_str
    )
    resp = requests.post(url, json={
        "model":    "dolphin-mistral:latest",
        "stream":   False,
        "messages": [{"role": "user", "content": prompt}]
    })
    resp.raise_for_status()
    data = resp.json()
    ai_report = (
        data["choices"][0]["message"]["content"].strip()
        if "choices" in data and data["choices"]
        else data.get("message",{}).get("content","_⚠️ AI returned empty response_")
    )

    md_lines += [
        "### AI Report",
        "",
        ai_report,
        ""
    ]

# 3) Write to file (pathlib ensures overwrite)
output_path = Path("castro_ai_summary.md")
output_path.write_text("\n".join(md_lines))
print(f"✅ Written summary for {last_dt or 'no date'} to {output_path}")

✅ Written summary for 2025-07-10 to castro_ai_summary.md


In [ ]:
%%bash
# don’t abort on error
set +e

# 1) Pull (ignore errors)
git pull --rebase origin main 2>&1

# 2) Stage everything
git add -A 2>&1

# 3) Commit & push only if there’s staged changes
if git diff --cached --quiet; then
  echo "No new changes, skipping commit"
else
  git commit -m "chore: update Castro AI summary" 2>&1 || true
  git push origin main 2>&1 || true
fi

# 4) Always exit zero so the notebook cell never errors
exit 0

In [ ]:
%run /mnt/scripts/py/update_report.py